# Bost-dtn Analysis

In [1]:
import os
import json
import numpy as np
from pathlib import Path

rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/bost-dtn"
# rootdir = "/home/eadhikarla/Desktop/Brian's Project/bost-dtn"

def traverse(path):
    fileList = []
    c = 0
    for root, directories, files in os.walk(path):
        for file in files:
            if file.endswith("json"):
                fileList.append(os.path.join(root,file))
                c+=1
    print(f"Total files: {c}")
    return fileList

==========================================
## bost-dtn (2021-06-09:14:35)
### 1. bbr2_p1

In [2]:
pscheduler_bbr2_p1  = os.path.join(rootdir, "2021-06-09:14:35/pscheduler_bbr2_p1")
filenames = traverse(pscheduler_bbr2_p1)

data_seg = []
tput_bbr2_p1 = []
key2 = 'bbr2_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        # The MongoDB JSON dump has one object per line, so this works for me.
        data = [json.loads(line) for line in open(f, 'r')]
        
        bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            
            if "interval" in d.keys() and j==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key2 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                bbr2_data_seg = data[j][key2]
                data_seg.append(bbr2_data_seg)

                throughput = ((bbr2_data_seg*mss*8)/(end_time-start_time))/1e9
                tput_bbr2_p1.append( throughput )
    
    except Exception as e:
        print(e)
        
print("Throughput")
print(f"Mean: {np.mean(tput_bbr2_p1)}  | Variance: {np.var(tput_bbr2_p1)}")
print("Data Segment")
print(f"Mean: {np.mean(data_seg):f}  | Variance: {np.var(data_seg):f}")

Total files: 105
Throughput
Mean: 6.016225764088099  | Variance: 6.998225016753158
Data Segment
Mean: 10593937.790476  | Variance: 53354882026455.648438


## bost-dtn (2021-06-09:14:35)
### 2. cubic_p1

In [3]:
pscheduler_cubic_p1 = os.path.join(rootdir, "2021-06-09:14:35/pscheduler_cubic_p1")
filenames = traverse(pscheduler_cubic_p1)

data_seg = []
tput_p1_cubic = []
key1 = 'cubic_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        # The MongoDB JSON dump has one object per line, so this works for me.
        data = [json.loads(line) for line in open(f, 'r')]
        
        cubic_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and j==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key1 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                cubic_data_seg = data[j][key1]
                data_seg.append(cubic_data_seg)

                throughput = ((cubic_data_seg*mss*8)/(end_time-start_time))/1e9
                tput_p1_cubic.append( throughput )
        # print(f"Throughput: {(throughput/1e9):.4f} Gbps/sec")
    
    except Exception as e:
        print(e)

print("Throughput")
print(f"Mean: {np.mean(tput_p1_cubic)}  | Variance: {np.var(tput_p1_cubic)}")
print("Data Segment")
print(f"Mean: {np.mean(data_seg)}  | Variance: {np.var(data_seg)}")

Total files: 105
Throughput
Mean: 6.523173995414669  | Variance: 8.671801878705109
Data Segment
Mean: 11281322.247619048  | Variance: 59836109647663.1


## bost-dtn (2021-06-09:14:35)
### 3. both_p16  -----> Fig. 4b. in paper

In [4]:
pscheduler_both_p16 = os.path.join(rootdir, "2021-06-09:14:35/pscheduler_both_p16")
filenames = traverse(pscheduler_both_p16)

data_seg_sum_cubic, data_seg_sum_bbr2 = [], []
tput_cubic_p16, tput_bbr2_p16 = [], []
key1, key2 = 'cubic_data_segs', 'bbr2_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in zip(range(len(data)),data):
            try:
                if "interval" in d.keys() and j==0:
                    start_time = d['interval']['time'] # Start time of the test
                if "streams" in d.keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size
                    
                    cubic_data_seg_list, bbr2_data_seg_list = [], []
                    for j in range(len(d['streams'])):
                        if "cubic" in d['streams'][j]['cc']:
                            cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                        elif "bbr2" in d['streams'][j]['cc']:
                            bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                    
                    data_seg_sum_cubic.append( sum(cubic_data_seg_list) )
                    throughput_cubic = ((sum(cubic_data_seg_list)*mss*8)/(end_time-start_time))/1e9
                    tput_cubic_p16.append( throughput_cubic )

                    data_seg_sum_bbr2.append( sum(bbr2_data_seg_list) )
                    throughput_bbr2 = ((sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time))/1e9
                    tput_bbr2_p16.append( throughput_bbr2 )

            except Exception as e:
                print(e)

    except Exception as e:
        print(e)

print("Throughput")
print(f"Cubic Mean: {np.mean(tput_cubic_p16)}  | Cubic Variance: {np.var(tput_cubic_p16)}")
print(f"Bbrv2 Mean: {np.mean(tput_bbr2_p16)}  | Bbrv2 Variance: {np.var(tput_bbr2_p16)}")

print("Data Segment")
print(f"Cubic Mean: {np.mean(data_seg_sum_cubic)}  | Cubic Variance: {np.var(data_seg_sum_cubic)}")
print(f"Bbrv2 Mean: {np.mean(data_seg_sum_bbr2)}  | Bbrv2 Variance: {np.var(data_seg_sum_bbr2)}")

Total files: 109
Throughput
Cubic Mean: 4.307491100831608  | Cubic Variance: 4.025419591687468
Bbrv2 Mean: 3.4211962376233123  | Bbrv2 Variance: 1.6866828401219744
Data Segment
Cubic Mean: 19844388.688073393  | Cubic Variance: 1702568119688929.5
Bbrv2 Mean: 16160038.23853211  | Bbrv2 Variance: 663028364607013.1


==========================================
## bost-dtn (2021-06-15:02:12)
### 1. bbr2_p1

In [5]:
pscheduler_bbr2_p1  = os.path.join(rootdir, "2021-06-15:02:12/pscheduler_bbr2_p1")
filenames = traverse(pscheduler_bbr2_p1)

data_seg = []
tput_bbr2_p1 = []
key2 = 'bbr2_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and j==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key2 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                bbr2_data_seg = data[j][key2]
                data_seg.append(bbr2_data_seg)

                throughput = ((bbr2_data_seg*mss*8)/(end_time-start_time))/1e9
                tput_bbr2_p1.append( throughput )
    
    except Exception as e:
        print(e)

print("Throughput")
print(f"Mean: {np.mean(tput_bbr2_p1)}  | Variance: {np.var(tput_bbr2_p1)}")
print("Data Segment")
print(f"Mean: {np.mean(data_seg)}  | Variance: {np.var(data_seg)}")

Total files: 45
Throughput
Mean: 6.731268177311756  | Variance: 8.108354896841512
Data Segment
Mean: 9662307.777777778  | Variance: 46214551992636.76


## bost-dtn (2021-06-15:02:12)
### 2. cubic_p1

In [6]:
pscheduler_cubic_p1 = os.path.join(rootdir, "2021-06-15:02:12/pscheduler_cubic_p1")
filenames = traverse(pscheduler_cubic_p1)

data_seg = []
tput_p1_cubic = []
key1 = 'cubic_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        # The MongoDB JSON dump has one object per line, so this works for me.
        data = [json.loads(line) for line in open(f, 'r')]
        
        cubic_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and j==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key1 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                cubic_data_seg = data[j][key1]
                data_seg.append(cubic_data_seg)

                throughput = ((cubic_data_seg*mss*8)/(end_time-start_time))/1e9
                tput_p1_cubic.append( throughput )
        # print(f"Throughput: {(throughput/1e9):.4f} Gbps/sec")
    
    except Exception as e:
        print(e)

print("Throughput")
print(f"Mean: {np.mean(tput_p1_cubic)}  | Variance: {np.var(tput_p1_cubic)}")
print("Data Segment")
print(f"Mean: {np.mean(data_seg)}  | Variance: {np.var(data_seg)}")

Total files: 47
Throughput
Mean: 6.718818006215081  | Variance: 9.18101181126119
Data Segment
Mean: 9722667.617021276  | Variance: 47272544227031.086


## bost-dtn (2021-06-15:02:12)
### 3. both_p16

In [7]:
pscheduler_both_p16 = os.path.join(rootdir, "2021-06-15:02:12/pscheduler_both_p16")
filenames = traverse(pscheduler_both_p16)

data_seg_sum_cubic, data_seg_sum_bbr2 = [], []
tput_cubic_p16, tput_bbr2_p16 = [], []
key1, key2 = 'cubic_data_segs', 'bbr2_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in zip(range(len(data)),data):
            try:
                if "interval" in d.keys() and j==0:
                    start_time = d['interval']['time'] # Start time of the test
                if "streams" in d.keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size
                    
                    cubic_data_seg_list, bbr2_data_seg_list = [], []
                    for j in range(len(d['streams'])):
                        if "cubic" in d['streams'][j]['cc']:
                            cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                        elif "bbr2" in d['streams'][j]['cc']:
                            bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                    
                    data_seg_sum_cubic.append( sum(cubic_data_seg_list) )
                    throughput_cubic = ((sum(cubic_data_seg_list)*mss*8)/(end_time-start_time))/1e9
                    tput_cubic_p16.append( throughput_cubic )

                    data_seg_sum_bbr2.append( sum(bbr2_data_seg_list) )
                    throughput_bbr2 = ((sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time))/1e9
                    tput_bbr2_p16.append( throughput_bbr2 )

            except Exception as e:
                print(e)

    except Exception as e:
        print(e)

print("Throughput")
print(f"Cubic Mean: {np.mean(tput_cubic_p16)}  | Cubic Variance: {np.var(tput_cubic_p16)}")
print(f"Bbrv2 Mean: {np.mean(tput_bbr2_p16)}  | Bbrv2 Variance: {np.var(tput_bbr2_p16)}")

print("Data Segment")
print(f"Cubic Mean: {np.mean(data_seg_sum_cubic)}  | Cubic Variance: {np.var(data_seg_sum_cubic)}")
print(f"Bbrv2 Mean: {np.mean(data_seg_sum_bbr2)}  | Bbrv2 Variance: {np.var(data_seg_sum_bbr2)}")

Total files: 50
Throughput
Cubic Mean: 2.3610244852519005  | Cubic Variance: 4.0516137939597145
Bbrv2 Mean: 5.80050963606564  | Bbrv2 Variance: 4.025665760087974
Data Segment
Cubic Mean: 8785820.88  | Cubic Variance: 218978255872263.2
Bbrv2 Mean: 34885604.12  | Bbrv2 Variance: 3816636072264945.5


==========================================
## bost-dtn (2021-06-16:02:12)
### 1. bbr2_p1

In [8]:
pscheduler_bbr2_p1  = os.path.join(rootdir, "2021-06-16:02:12/pscheduler_bbr2_p1")
filenames = traverse(pscheduler_bbr2_p1)

data_seg = []
tput_bbr2_p1 = []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and j==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key2 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                bbr2_data_seg = data[j][key2]
                data_seg.append( bbr2_data_seg )

                throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)/1e9
                tput_bbr2_p1.append( throughput )
    
    except Exception as e:
        print(e)

print("Throughput")
print(f"Mean: {np.mean(tput_bbr2_p1)}  | Variance: {np.var(tput_bbr2_p1)}")
print("Data Segment")
print(f"Mean: {np.mean(data_seg)}  | Variance: {np.var(data_seg)}")

Total files: 55
Throughput
Mean: 6.331755373365051  | Variance: 8.16012090866518
Data Segment
Mean: 11637878.672727272  | Variance: 59359636395545.71


## bost-dtn (2021-06-16:02:12)
### 2. cubic_p1

In [9]:
pscheduler_cubic_p1 = os.path.join(rootdir, "2021-06-16:02:12/pscheduler_cubic_p1")
filenames = traverse(pscheduler_cubic_p1)

data_seg = []
tput_p1_cubic = []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        # The MongoDB JSON dump has one object per line, so this works for me.
        data = [json.loads(line) for line in open(f, 'r')]
        
        cubic_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and j==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key1 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                cubic_data_seg = data[j][key1]
                data_seg.append( cubic_data_seg )

                throughput = (cubic_data_seg*mss*8)/(end_time-start_time)/1e9
                tput_p1_cubic.append( throughput )
    
    except Exception as e:
        print(e)

print("Throughput")
print(f"Mean: {np.mean(tput_p1_cubic)}  | Variance: {np.var(tput_p1_cubic)}")
print("Data Segment")
print(f"Mean: {np.mean(data_seg)}  | Variance: {np.var(data_seg)}")

Total files: 54
Throughput
Mean: 6.349940200589246  | Variance: 9.532853858764945
Data Segment
Mean: 11014398.944444444  | Variance: 62556039019839.65


## bost-dtn (2021-06-16:02:12)
### 3. both_p16

In [ ]:
pscheduler_both_p16 = os.path.join(rootdir, "2021-06-16:02:12/pscheduler_both_p16")
filenames = traverse(pscheduler_both_p16)

data_seg_sum_cubic, data_seg_sum_bbr2 = [], []
tput_cubic_p16, tput_bbr2_p16 = [], []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in zip(range(len(data)),data):
            try:
                if "interval" in d.keys() and j==0:
                    start_time = d['interval']['time'] # Start time of the test
                if "streams" in d.keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size
                    
                    cubic_data_seg_list, bbr2_data_seg_list = [], []
                    for j in range(len(d['streams'])):
                        if "cubic" in d['streams'][j]['cc']:
                            cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                        elif "bbr2" in d['streams'][j]['cc']:
                            bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                    
                    data_seg_sum_cubic.append( sum(cubic_data_seg_list) )
                    throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                    tput_cubic_p16.append( throughput_cubic )

                    data_seg_sum_bbr2.append( sum(bbr2_data_seg_list) )
                    throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                    tput_bbr2_p16.append( throughput_bbr2 )

            except Exception as e:
                print(e)

    except Exception as e:
        print(e)

print("Throughput")
print(f"Cubic Mean: {np.mean(tput_cubic_p16)}  | Cubic Variance: {np.var(tput_cubic_p16)}")
print(f"Bbrv2 Mean: {np.mean(tput_bbr2_p16)}  | Bbrv2 Variance: {np.var(tput_bbr2_p16)}")

print("Data Segment")
print(f"Cubic Mean: {np.mean(data_seg_sum_cubic)}  | Cubic Variance: {np.var(data_seg_sum_cubic)}")
print(f"Bbrv2 Mean: {np.mean(data_seg_sum_bbr2)}  | Bbrv2 Variance: {np.var(data_seg_sum_bbr2)}")

# Buffer Analysis